In [4]:
from pathlib import Path

## 形態素解析結果の読み込み

In [18]:
res_mecab = Path("neko.txt.mecab")
sentences = []
morphs = []

with res_mecab.open() as file:
  for line in file:
    line = line.strip()
    if line != "EOS":
      fields = line.split("\t")
      if len(fields) != 2 or fields[0] == "": continue
      col = fields[1].split(",")
      morph = {"surface": fields[0], "base": col[6], "pos": col[0], "pos1": col[1]}
      morphs.append(morph)
    else:
      sentences.append(morphs)
      morphs = []
for morph in sentences[2]:
  print(morph)

{'surface': '吾輩', 'base': '吾輩', 'pos': '名詞', 'pos1': '代名詞'}
{'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'}
{'surface': '猫', 'base': '猫', 'pos': '名詞', 'pos1': '一般'}
{'surface': 'で', 'base': 'だ', 'pos': '助動詞', 'pos1': '*'}
{'surface': 'ある', 'base': 'ある', 'pos': '助動詞', 'pos1': '*'}
{'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}


## 動詞と動詞の基本形の抜き出し

In [19]:
verbs = set(morph["surface"] for morphs in sentences for morph in morphs
            if morph["pos"] == "動詞")
verb_bases = set(morph["base"] for morphs in sentences for morph in morphs
                 if morph["pos"] == "動詞")

for i, verb in enumerate(verbs):
  if i == 5: break
  print(verb)

笑う
よす
安んずる
収め
借り


In [17]:
# 笑う
# よす
# 安んずる
# 収め
# 借り

{'surface': '吾輩', 'base': '吾輩', 'pos': '名詞', 'pos1': '代名詞'}
{'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'}
{'surface': '猫', 'base': '猫', 'pos': '名詞', 'pos1': '一般'}
{'surface': 'で', 'base': 'だ', 'pos': '助動詞', 'pos1': '*'}
{'surface': 'ある', 'base': 'ある', 'pos': '助動詞', 'pos1': '*'}
{'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}


In [ ]:
# {'surface': '吾輩', 'base': '吾輩', 'pos': '名詞', 'pos1': '代名詞'}
# {'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'}
# {'surface': '猫', 'base': '猫', 'pos': '名詞', 'pos1': '一般'}
# {'surface': 'で', 'base': 'だ', 'pos': '助動詞', 'pos1': '*'}
# {'surface': 'ある', 'base': 'ある', 'pos': '助動詞', 'pos1': '*'}
# {'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}